In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import random
from tensorflow import keras
from keras.datasets import fashion_mnist

In [ ]:
# carregando datasets do keras
#from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [ ]:
# obtendo informações das imagens (resolucao) e dos rótulos (número de classes)
img_lin, img_col = x_train.shape[1], x_train.shape[2]
num_classes = len(np.unique(y_train))
print(x_train.shape)
print('Classes: ', num_classes)

(60000, 28, 28)
Classes:  10


Temos portando 60 mil exemplos de treinamento

Imagens de tamanho $28\times 28$ e 10 classes.

Agora, o passo ideal é pré-processar as imagens para facilitar o processo de otimização das redes neurais:

1. Normalizar seus valores (para o intervalo 0-1)
2. Alterar a codificação das classes (para one-hot-encoding): teremos 10 neurônios na saída, não apenas 1

In [ ]:
# dividir por 255 para obter normalizacao
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

print("Antes da transformação: ", y_train[0])
# transformar categorias em one-hot-encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("Após transformação: ", y_train[0])

Antes da transformação:  9
Após transformação:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


Por fim, vamos padronizar o formato de entrada em termos das dimensões (shape) do numpy array de entrada.

Isso vai permitir que o mesmo código utilize diferentes bases de dados, com um ou mais canais de cor

In [ ]:
# verifica imagens da base de dados tem 3 canais (RGB) ou apenas 1 (escala de cinza)
if (len(x_train.shape) == 3):
      n_channels = 1
else:
      n_channels = x_train.shape[3]

# re-formata o array de forma a encontrar o formato da entrada (input_shape)
# se a dimensão dos canais vem primeiro ou após a imagem
if keras.backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], n_channels, img_lin, img_col)
    x_test = x_test.reshape(x_test.shape[0], n_channels, img_lin, img_col)
    input_shape = (n_channels, img_lin, img_col)
else:
    x_train = x_train.reshape(x_train.shape[0], img_lin, img_col, n_channels)
    x_test = x_test.reshape(x_test.shape[0], img_lin, img_col, n_channels)
    input_shape = (img_lin, img_col, n_channels)

print("Shape: ", input_shape)

Shape:  (28, 28, 1)


### Tudo pronto, agora vamos começar


In [ ]:
def my_cnn():
    CNN = keras.Sequential()
    CNN.add(keras.layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=input_shape))
    CNN.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    CNN.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu'))
    CNN.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    CNN.add(keras.layers.GlobalAveragePooling2D())
    CNN.add(keras.layers.Dense(64, activation='relu'))
    CNN.add(keras.layers.Dropout(0.25))
    CNN.add(keras.layers.Dense(10, activation='softmax'))

    return CNN

Definindo sementes e um subconjunto menor para testarmos


In [ ]:
# as sementes ajudam a ter resultados reproduzíveis
from numpy.random import seed
seed(1)
random.set_seed(2)

x_train_subset = x_train[:12000]
y_train_subset = y_train[:12000]
x_val_subset = x_test[:8000]
y_val_subset = y_test[:8000]
batch_size = 20
epochs = 10

### 1) Modelo convencional com parâmetros padrão (learning rate fixo)

In [ ]:
CNN1 = my_cnn()
CNN1.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy'])

In [ ]:
import numpy as np

def run_experiment(x_t, y_t, x_v, y_v, batch_size, epochs):
  """
  Realiza o experimento de treinamento e avaliação da rede convolucional

  Args:
    x_t: Conjunto de treinamento reduzido
    y_t: Rótulos do conjunto de treinamento reduzido
    x_v: Conjunto de validação
    y_v: Rótulos do conjunto de validação
    batch_size: Tamanho do lote
    epochs: Número de épocas

  Returns:
    Lista com as acurácias no conjunto de treinamento e validação
  """

  train_mean = []
  val_mean = []
  for _ in range(5):
    hist = CNN1.fit(x_t, y_t,
                    batch_size=batch_size,
                    epochs=epochs, validation_data=(x_v, y_v))
    train_mean.append(np.mean(hist.history["accuracy"]))
    val_mean.append(np.mean(hist.history["val_accuracy"]))

  return train_mean, val_mean

train_mean, val_mean = run_experiment(x_train_subset, y_train_subset, x_val_subset, y_val_subset, batch_size, epochs)

print("Acurácia média no conjunto de treinamento:", train_mean)
print("Acurácia média no conjunto de validação:", val_mean)

Epoch 1/10
600/600 [==============================] - 18s 6ms/step - loss: 1.0922 - accuracy: 0.5791 - val_loss: 0.7361 - val_accuracy: 0.7151
Epoch 2/10
600/600 [==============================] - 3s 5ms/step - loss: 0.7060 - accuracy: 0.7341 - val_loss: 0.5552 - val_accuracy: 0.8006
Epoch 3/10
600/600 [==============================] - 4s 7ms/step - loss: 0.5717 - accuracy: 0.7925 - val_loss: 0.4917 - val_accuracy: 0.8219
Epoch 4/10
600/600 [==============================] - 4s 6ms/step - loss: 0.4941 - accuracy: 0.8199 - val_loss: 0.4692 - val_accuracy: 0.8324
Epoch 5/10
600/600 [==============================] - 4s 6ms/step - loss: 0.4553 - accuracy: 0.8347 - val_loss: 0.5457 - val_accuracy: 0.8105
Epoch 6/10
600/600 [==============================] - 3s 6ms/step - loss: 0.4118 - accuracy: 0.8528 - val_loss: 0.4429 - val_accuracy: 0.8416
Epoch 7/10
600/600 [==============================] - 4s 7ms/step - loss: 0.3884 - accuracy: 0.8579 - val_loss: 0.4102 - val_accuracy: 0.8562
Epoch

In [ ]:
print("Acurácia média enre os conjuntos de treinamento:", np.mean(train_mean))
print("Acurácia média enre os conjuntos de validação:", np.mean(val_mean))

Acurácia média enre os conjuntos de treinamento: 0.9185116648674011
Acurácia média enre os conjuntos de validação: 0.8639374983310699
